# Demographics of Los Angeles County 

This notebook covers the demographics for Los Angeles County at the census block level. The data will be compared to the locations of publicly available toilets throughout the county to better understand some of the demographic characteristics of publicly avialble toilets and washing stations. 

In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

In [2]:
race = pd.read_csv('ACSDP5Y2019.DP05_data_with_overlays_2022-06-02T223045.csv')
race.head()

C:\Users\User\AppData\Local\Temp\ipykernel_7092\3316682979.py:1: DtypeWarning: Columns (0,1,2,4,5,8,9,16,17,20,21,24,25,28,29,32,33,36,37,40,41,44,45,48,49,52,53,56,57,60,61,64,65,72,73,76,77,80,81,84,85,88,89,92,93,96,97,98,100,101,104,105,112,113,114,116,117,120,121,128,129,130,132,133,136,137,140,141,144,145,148,149,152,153,156,157,160,161,164,165,168,169,172,173,176,177,180,181,184,185,188,189,192,193,196,197,200,201,204,205,208,209,212,213,216,217,220,221,224,225,228,229,232,233,236,237,240,241,244,245,248,249,250,252,253,256,257,260,261,264,265,268,269,272,273,276,277,278,280,281,284,285,288,289,292,293,296,297,300,301,304,305,308,309,312,313,316,317,320,321,324,325,328,329,332,333,336,337,340,341,344,345,346,348,349,352,353) have mixed types. Specify dtype option on import or set low_memory=False.
  race = pd.read_csv('ACSDP5Y2019.DP05_data_with_overlays_2022-06-02T223045.csv')


,DP05_0001E,DP05_0001M,DP05_0001PE,DP05_0001PM,DP05_0002E,DP05_0002M,DP05_0002PE,DP05_0002PM,DP05_0003E,DP05_0003M,...,DP05_0088E,DP05_0088M,DP05_0088PE,DP05_0088PM,DP05_0089E,DP05_0089M,DP05_0089PE,DP05_0089PM,GEO_ID,NAME
0,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population...,Percent!!SEX AND AGE!!Total population!!Male,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population...,...,"Estimate!!CITIZEN, VOTING AGE POPULATION!!Citi...","Margin of Error!!CITIZEN, VOTING AGE POPULATIO...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citi...","Margin of Error!!CITIZEN, VOTING AGE POPULATIO...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...",id,Geographic Area Name
1,4283,443,4283,(X),2040,285,47.6,3.6,2243,250,...,1301,216,44.7,4.6,1611,193,55.3,4.6,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali..."
2,3405,334,3405,(X),1626,218,47.8,4.9,1779,254,...,1446,190,49.7,5.0,1466,190,50.3,5.0,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali..."
3,6347,484,6347,(X),3350,343,52.8,3.5,2997,314,...,1953,251,52.5,5.0,1765,260,47.5,5.0,1400000US06037101210,"Census Tract 1012.10, Los Angeles County, Cali..."
4,3702,276,3702,(X),1717,197,46.4,3.6,1985,184,...,1181,160,47.9,3.0,1283,137,52.1,3.0,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali..."
